In [39]:
pip install dash

  Using cached dash-2.14.1-py3-none-any.whl.metadata (11 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
  Using cached ansi2html-1.8.0-py3-none-any.whl (16 kB)
Using cached dash-2.14.1-py3-none-any.whl (10.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [4]:
filepath = '/Users/jacquelinethibault/Desktop/ResponseApproximation/fa23urap/test3.txt' #result from thebeTest
df = pd.read_csv(filepath, delimiter = ',')

In [5]:
df.head()

,Unnamed: 0,Event ID,Energy,Theta,Phi,Scatter Angle,Path Length (cm),Theta Bin,Phi Bin,Scatter Angle Bin
0,0,1,509.8545,95.111090,-79.695154,22.103606,1.122497,96.630738,-74.931512,23.016826
1,1,2,510.7631,90.000000,-45.000000,40.497715,0.282843,90.000000,-45.000000,41.500863
2,2,3,509.1930,165.046589,33.690068,82.068576,2.794638,180.000000,43.264295,83.771117
3,3,4,511.8270,103.109018,-152.241459,63.178452,2.204541,103.240520,-137.726311,63.850514
4,4,6,510.1480,170.251968,-104.036243,68.882363,2.435159,180.000000,-103.348727,72.330605


exploring how far from avg the number of items in each bin are.

In [6]:
meanitems=df.groupby('Phi Bin').count().iloc[:,0].mean()#.append(df.groupby('Phi Bin').count().iloc[9:,0]).mean()
display(meanitems - df.groupby('Phi Bin').count().iloc[:,0])
display(meanitems)

Phi Bin
-165.173520     14403.947368
-137.726311     14087.947368
-124.159695     14704.947368
-113.198591     14391.947368
-103.348727     14428.947368
-90.000000    -261889.052632
-74.931512      16987.947368
-64.536655      14236.947368
-55.007980      13796.947368
-45.000000       4896.947368
-23.198591      24621.947368
-0.000000     -172385.052632
 14.036243     199487.947368
 43.264295      16197.947368
 66.801409      14357.947368
 90.000000     -54140.052632
 120.963757     82742.947368
 148.671307     14659.947368
 179.530373     14408.947368
Name: Unnamed: 0, dtype: float64

288114.94736842107

dropped the weird 90 and 0 data

In [54]:
df_dropped = df[df['Theta'] != 90][df['Phi'] != -90][df['Phi'] != 0][df['Phi']!=90]
meanitems=df_dropped.groupby('Phi Bin').count().iloc[:,0].mean()#.append(df.groupby('Phi Bin').count().iloc[9:,0]).mean()
display(meanitems - df_dropped.groupby('Phi Bin').count().iloc[:,0])

/var/folders/0p/p_3sbhs92f1cbn5q60vgrkxw0000gn/T/ipykernel_44923/2273803363.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Phi Bin
-165.173520     13536.578947
-137.726311    -32680.421053
-124.159695    -18678.421053
-113.198591    -27911.421053
-103.348727    -32127.421053
-90.000000     -14577.421053
-74.931512     -35619.421053
-64.536655     -34072.421053
-55.007980     -27545.421053
-45.000000     -25646.421053
-23.198591     -22711.421053
-0.000000       72687.578947
 14.036243     139273.578947
 43.264295     -28445.421053
 66.801409     -13414.421053
 90.000000     101299.578947
 120.963757     34117.578947
 148.671307    -16900.421053
 179.530373    -30584.421053
Name: Unnamed: 0, dtype: float64

In [62]:
#df.sort_values(by='Scatter Angle Bin').groupby('Scatter Angle Bin').count().index
np.sort(df['Scatter Angle Bin'].unique())

array([ 18.86019372,  23.01682591,  26.9141888 ,  30.64293167,
        34.24417553,  37.8168135 ,  41.50086348,  45.30421865,
        49.33871276,  53.73924469,  58.44100575,  63.85051434,
        72.33060542,  83.7711166 ,  94.5099787 , 103.50121702,
       113.82857547, 125.75410468, 140.42414372, 179.91867374])

In [66]:
def addLowerBound(newdf):

    phi = np.sort(newdf['Phi Bin'].unique()) #-180 to 180
    theta = np.sort(newdf['Theta Bin'].unique()) # 0 to 180
    sc = np.sort(newdf['Scatter Angle Bin'].unique()) #0 to 180
    vardict = {0: phi, 1: theta, 2: sc}
    stringdict = {0: 'Phi Bounds', 1: 'Theta Bounds', 2: 'Scatter Angle Bounds'}
    oldstringdict = {0: 'Phi Bin', 1: 'Theta Bin', 2: 'Scatter Angle Bin'}
    dictmaps = {}
    for i in range(3):
        var = vardict[i]
        varstring = stringdict[i]
        bounded = []
        lowerbound = 0
        if varstring == 'Phi Bounds':
            lowerbound = -180
        for upperbound in var:
            bounded.append(f'{lowerbound}-{upperbound}')
            dictmaps[upperbound] = bounded[-1]
            lowerbound = upperbound
        newdf[varstring] = [dictmaps[value] for value in newdf[oldstringdict[i]]]
    return newdf

df=addLowerBound(df)
    

In [45]:
df.groupby(['Theta Bin', 'Phi Bin', 'Scatter Angle Bin']).count().head(50)

Unnamed: 0  Event ID  Energy  Theta  \
Theta Bin Phi Bin     Scatter Angle Bin                                        
27.791306 -165.173520 18.860194                 160       160     160    160   
                      23.016826                 162       162     162    162   
                      26.914189                 190       190     190    190   
                      30.642932                 255       255     255    255   
                      34.244176                 320       320     320    320   
                      37.816813                 411       411     411    411   
                      41.500863                 549       549     549    549   
                      45.304219                 587       587     587    587   
                      49.338713                 729       729     729    729   
                      53.739245                 578       578     578    578   
                      58.441006                 397       397     397    397   
                      63.850514                 371       371     371    371   
                      72.330605                 436       436     436    436   
                      83.771117                 569       569     569    569   
                      94.509979                1809      1809    1809   1809   
                      103.501217               6973      6973    6973   6973   
                      113.828575               5429      5429    5429   5429   
                      125.754105               1498      1498    1498   1498   
                      140.424144                456       456     456    456   
                      179.918674                196       196     196    196   
          -137.726311 18.860194                  66        66      66     66   
                      23.016826                  84        84      84     84   
                      26.914189                  72        72      72     72   
                      30.642932                 116       116     116    116   
                      34.244176                 153       153     153    153   
                      37.816813                 212       212     212    212   
                      41.500863                 254       254     254    254   
                      45.304219                 311       311     311    311   
                      49.338713                 301       301     301    301   
                      53.739245                 231       231     231    231   
                      58.441006                 192       192     192    192   
                      63.850514                 191       191     191    191   
                      72.330605                 258       258     258    258   
                      83.771117                 434       434     434    434   
                      94.509979                3648      3648    3648   3648   
                      103.501217               3601      3601    3601   3601   
                      113.828575                595       595     595    595   
                      125.754105                145       145     145    145   
                      140.424144                 71        71      71     71   
                      179.918674                 25        25      25     25   
          -124.159695 18.860194                  70        70      70     70   
                      23.016826                 102       102     102    102   
                      26.914189                 113       113     113    113   
                      30.642932                 138       138     138    138   
                      34.244176                 211       211     211    211   
                      37.816813                 248       248     248    248   
                      41.500863                 240       240     240    240   
                      45.304219                 272       272     272    272   
                      49.338713          

In [9]:
justbins_df=df[['Theta Bin', 'Phi Bin', 'Scatter Angle Bin']]
justbins_df

,Theta Bin,Phi Bin,Scatter Angle Bin
0,96.630738,-74.931512,23.016826
1,90.000000,-45.000000,41.500863
2,180.000000,43.264295,83.771117
3,103.240520,-137.726311,63.850514
4,180.000000,-103.348727,72.330605
...,...,...,...
5474179,61.094381,-45.000000,58.441006
5474180,61.094381,-124.159695,103.501217
5474181,90.000000,-55.007980,23.016826
5474182,88.538861,-45.000000,41.500863


In [30]:
justbins_df_sampled= justbins_df.sample(n=500)
df_sampled = df.sample(n=500) #[df['Scatter Angle'] < 60][df['Scatter Angle'] > 40]
df_sampled

,Unnamed: 0,Event ID,Energy,Theta,Phi,Scatter Angle,Path Length (cm),Theta Bin,Phi Bin,Scatter Angle Bin
5214647,5214647,9526045,510.9205,90.000000,-71.565051,15.041408,0.632456,90.000000,-64.536655,18.860194
1430181,1430181,2612600,509.6070,114.094843,116.565051,146.530731,0.244949,114.937983,120.963757,179.918674
2993351,2993351,5467613,511.9140,12.528808,90.000000,113.389236,0.921954,27.791306,90.000000,113.828575
1116742,1116742,2040065,511.5890,135.000000,-90.000000,45.419971,0.707107,140.205935,-90.000000,49.338713
2312695,2312695,4224114,507.9860,96.655545,103.495733,49.175111,2.588436,103.240520,120.963757,49.338713
...,...,...,...,...,...,...,...,...,...,...
64946,64946,117960,509.7750,90.000000,158.198591,105.664770,0.538516,90.000000,179.530373,113.828575
5394374,5394374,9853886,510.1930,74.498640,-56.309932,44.065965,0.374166,74.769353,-55.007980,45.304219
4424361,4424361,8082782,510.5380,46.508481,108.434949,130.315305,0.435890,53.225219,120.963757,140.424144
3036459,3036459,5546789,511.1320,60.982859,-56.309932,48.744494,0.412311,61.094381,-55.007980,49.338713


In [35]:
#px.scatter_3d(df_sampled, x='Theta Bin', y='Phi Bin', z='Scatter Angle Bin')
px.scatter_3d(df_sampled, x='Theta', y='Phi', z='Scatter Angle', color = 'Scatter Angle Bin', size = 'Path Length (cm)')

In [43]:

app = Dash(__name__)

app.layout = html.Div([
    html.H4('cds filtered by scatter angle bin'),
    dcc.Graph(id="graph"),
    html.P("Scatter Angle:"),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=180, step=1,
        marks={0: '0', 180: '180'},
        value=[0, 180]
    ),
])

@app.callback(
    Output("graph", "figure"), 
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    df = df_sampled
    low, high = slider_range
    mask = (df['Scatter Angle'] > low) & (df['Scatter Angle'] < high)

    fig = px.scatter_3d(df[mask], 
        x='Theta', y='Phi', z='Scatter Angle',
        color="Scatter Angle Bin" , size = 'Path Length (cm)')
    return fig

app.run_server(debug=True)